In [1]:
# load the library
from bs4 import BeautifulSoup
import urllib2, requests
import pandas as pd

df = pd.DataFrame()   # create a new data frame
pd.set_option('max_colwidth',500)    # to remove column limit (default is 50. Otherwise, we'll lose some info)


# indeed.com's query url
base_url = 'http://www.indeed.com/jobs?q=data+scientist&jt=fulltime&sort='
sort_by = 'date'          # sort by data
start_from = '&start='    # start page number

In [2]:
for page in range(1,11): # page from 1 to 100 
    page = (page-1) * 10  
    url = "%s%s%s%d" % (base_url, sort_by, start_from, page) # get full url 
    contents = BeautifulSoup(urllib2.urlopen(url), "lxml") 

    contentsElements = contents.find_all('div', attrs={'class' : '  row  result'}) 
    # each row contains 1 job
    
    # extract job information
    for elem in contentsElements: 
        try:
            company = elem.find('span', attrs={'itemprop':'name'}).getText().strip() #strips off whitespace 
        except AttributeError:
            company = '-'
        job_title = elem.find('a', attrs={'class':'turnstileLink'}).attrs['title']
        home_url = "http://www.indeed.com"
        job_link = "%s%s" % (home_url,elem.find('a').get('href'))
        job_addr = elem.find('span', attrs={'itemprop':'addressLocality'}).getText()
        job_posted = elem.find('span', attrs={'class': 'date'}).getText()


        # add a record of job info to our data frame
        df = df.append({'company': company, 
                        'job_title': job_title, 
                        'job_link': job_link, 
                        'job_posted': job_posted,
                        'job_location': job_addr
                       }, ignore_index=True)

df    

,company,job_link,job_location,job_posted,job_title
0,Lockheed Martin,http://www.indeed.com/rc/clk?jk=5fe8bfadf6abdb68&fccid=aeb15e43a6800b9d,"Washington, DC",Just posted,Data Scientist Analyst
1,Leidos,http://www.indeed.com/rc/clk?jk=cf7e3c9ebf7f2a73&fccid=8765a4045377753a,"Washington, DC",Just posted,Quantitative Statistical Analyst
2,Expedia,http://www.indeed.com/rc/clk?jk=c5940df8261b9a0c&fccid=3169292148c15eab,"Bellevue, WA 98004 (Downtown area)",Just posted,Data Scientist
3,"Xyant Technology, Inc.",http://www.indeed.com/rc/clk?jk=9c92016b05cb73f7&fccid=9d722b0c888b17b1,"Lowell, AR",Just posted,Data Engineer
4,Brillio,http://www.indeed.com/rc/clk?jk=b1da5587940342a3&fccid=fac5a2b8f07c805e,"Redmond, WA",Just posted,Data Scientist
5,S&P GLOBAL MARKET INTELLIGENCE,http://www.indeed.com/rc/clk?jk=bd7252a39f86b195&fccid=2c2386465bad2305,"New York, NY 10002 (Lower East Side area)",Just posted,"Research Analyst, Financial Technology"
6,RangTech,http://www.indeed.com/rc/clk?jk=d7b623d5f3ce2069&fccid=f924423dab22b92a,"Township of Piscataway, NJ",Just posted,Data Scientist
7,SUNY Downstate Medical Center,http://www.indeed.com/rc/clk?jk=3fef309756ed24f5&fccid=ff707404f3ba0ae9,"Brooklyn, NY",Just posted,DATA MANAGER/STATISTICIAN - PART-TME
8,Department of Defense,http://www.indeed.com/rc/clk?jk=861f224e426142ab&fccid=25801a556a902df9,"Fort McNair, DC",Just posted,Research Analyst
9,BNY Mellon,http://www.indeed.com/rc/clk?jk=b2a21ac11cf91f9a&fccid=b4048be2884af072,"New York, NY",Just posted,Jr Quantitative Risk Analyst


In [3]:
# check one entry to verify contents
print df.iloc[6]

company                                                                        RangTech
job_link        http://www.indeed.com/rc/clk?jk=d7b623d5f3ce2069&fccid=f924423dab22b92a
job_location                                                 Township of Piscataway, NJ
job_posted                                                                  Just posted
job_title                                                                Data Scientist
Name: 6, dtype: object


In [4]:
# Save the result to CSV
df.to_csv('./indeed_companies.csv', encoding='utf-8')